# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](https://course.cse.ust.hk/comp4901j/Password_Only/programs/assignment1/index.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [1]:
import random
import numpy as np
from data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


In [23]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.409200
sanity check: 2.302585


## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** 
- Since there we are initializing our Weight Matrix with random numbers selected from a uniform probability distribution, we might guess that all of our initial models will have similar losses. When doing the mean among all the losses we encounter the fact the it should be similar to the probability of randomly picking the correct class between the 10 e.g p=1/10=0.1 where all classes have the same probability of being selected.


In [25]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 0.113538 analytic: 0.113538, relative error: 4.830896e-08
numerical: 0.392294 analytic: 0.392294, relative error: 1.516432e-07
numerical: 1.609479 analytic: 1.609479, relative error: 6.366775e-09
numerical: 0.682891 analytic: 0.682891, relative error: 4.527591e-08
numerical: 0.872445 analytic: 0.872445, relative error: 1.026414e-10
numerical: -1.729975 analytic: -1.729975, relative error: 3.222360e-08
numerical: -0.791450 analytic: -0.791450, relative error: 2.666005e-09
numerical: 1.128486 analytic: 1.128486, relative error: 3.478646e-08
numerical: 0.884497 analytic: 0.884497, relative error: 1.278950e-07
numerical: 0.817386 analytic: 0.817386, relative error: 8.189523e-08
numerical: 1.442213 analytic: 1.442213, relative error: 3.856776e-08
numerical: -1.359357 analytic: -1.359357, relative error: 1.250883e-08
numerical: 0.977326 analytic: 0.977326, relative error: 4.167274e-10
numerical: 1.188429 analytic: 1.188429, relative error: 7.225649e-09
numerical: -2.768959 analyti

In [48]:
from cs231n.classifiers.softmax import softmax_loss_vectorized
vec_loss,vec_grad = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)

In [52]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.409200e+00 computed in 36.380885s
vectorized loss: 2.409200e+00 computed in 0.005466s
Loss difference: 0.000000
Gradient difference: 0.000000


In [55]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax

results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
pass

for i in np.linspace(learning_rates[0],learning_rates[1],num=4):
    for j in np.linspace(regularization_strengths[0],regularization_strengths[1],num=4):
        
        model = Softmax()
        model.train(X_train,y_train,learning_rate=i,reg=j,num_iters=800,verbose=False) #train model
        
        results[(i,j)] = (np.mean(model.predict(X_train)==y_train),np.mean(model.predict(X_val)==y_val)) #Generate and Store Results
        
        if results[(i,j)][1] > best_val:   # Check for possible best new score
            best_val = results[(i,j)][1]
            best_softmax = model
        else:
            pass
        
pass
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.318918 val accuracy: 0.329000
lr 1.000000e-07 reg 3.333333e+04 train accuracy: 0.331143 val accuracy: 0.346000
lr 1.000000e-07 reg 4.166667e+04 train accuracy: 0.331000 val accuracy: 0.344000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.327694 val accuracy: 0.336000
lr 2.333333e-07 reg 2.500000e+04 train accuracy: 0.350061 val accuracy: 0.367000
lr 2.333333e-07 reg 3.333333e+04 train accuracy: 0.341163 val accuracy: 0.358000
lr 2.333333e-07 reg 4.166667e+04 train accuracy: 0.330061 val accuracy: 0.348000
lr 2.333333e-07 reg 5.000000e+04 train accuracy: 0.331918 val accuracy: 0.346000
lr 3.666667e-07 reg 2.500000e+04 train accuracy: 0.352735 val accuracy: 0.364000
lr 3.666667e-07 reg 3.333333e+04 train accuracy: 0.337939 val accuracy: 0.347000
lr 3.666667e-07 reg 4.166667e+04 train accuracy: 0.335122 val accuracy: 0.359000
lr 3.666667e-07 reg 5.000000e+04 train accuracy: 0.324837 val accuracy: 0.329000
lr 5.000000e-07 reg 2.500000

In [56]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.360000


In [57]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])